# Lab 5 – Bias–Variance Tradeoff

This notebook implements all the steps from **lab5.md** using the `AirQualityUCI.csv` dataset.
Run the single code cell below after placing `AirQualityUCI.csv` in the same folder.


In [ ]:
# CE49X – Lab 5: Bias–Variance Tradeoff using the Air Quality Dataset
# ---------------------------------------------------------------
# 1. Load & clean AirQualityUCI data
# 2. Use T, RH, AH to predict CO(GT)
# 3. Train polynomial regression models (degrees 1–10)
# 4. Plot training & testing error vs degree (bias–variance tradeoff)
# 5. BONUS: Use k-fold cross-validation to estimate generalization error
# 6. Step 4: Discussion answers printed at the end

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

# ---------------------------------------------------------------
# Step 1 — Load and Prepare the Data
# ---------------------------------------------------------------

# Use the CSV version (Excel is also available but not needed here)
data_path = "AirQualityUCI.csv"

# UCI file uses semicolon as separator and has an empty last column
df_raw = pd.read_csv(data_path, sep=';')

# Drop completely empty columns (e.g., unnamed last column)
df_raw = df_raw.dropna(axis=1, how='all')

# Replace -200 (missing data code) with NaN
df = df_raw.replace(-200, np.nan)

# Select relevant columns
features = ['T', 'RH', 'AH']
target = 'CO(GT)'

# Keep only rows where all needed columns are present
df_model = df[features + [target]].dropna()

# Define X (features) and y (target)
X = df_model[features].values
y = df_model[target].values

print("Shape of cleaned feature matrix X:", X.shape)
print("Shape of target vector y:", y.shape)

# Train–test split (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

# ---------------------------------------------------------------
# Step 2 — Fit Models of Increasing Complexity (degrees 1–10)
# ---------------------------------------------------------------

degrees = list(range(1, 11))
train_errors = []
test_errors = []

for d in degrees:
    # Create polynomial features of degree d
    poly = PolynomialFeatures(degree=d, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Fit Linear Regression on the transformed features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predictions
    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)

    # Mean Squared Error (MSE)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)

    train_errors.append(mse_train)
    test_errors.append(mse_test)

    print(f"Degree {d:2d} | Train MSE: {mse_train:.4f} | Test MSE: {mse_test:.4f}")

# Find the degree with minimum test error
best_idx = int(np.argmin(test_errors))
best_degree = degrees[best_idx]
print("\nBest polynomial degree (min test MSE):", best_degree)

# ---------------------------------------------------------------
# Step 3 — Plot the Validation Curve (Bias–Variance Tradeoff)
# ---------------------------------------------------------------

plt.figure(figsize=(9, 5))
plt.plot(degrees, train_errors, marker='o', label='Training Error (MSE)')
plt.plot(degrees, test_errors, marker='s', label='Testing Error (MSE)')

plt.xlabel('Model Complexity (Polynomial Degree)')
plt.ylabel('Mean Squared Error')
plt.xticks(degrees)
plt.title('Bias–Variance Tradeoff for Polynomial Regression on Air Quality Data')
plt.grid(True)

# Mark optimal complexity
plt.axvline(best_degree, linestyle='--', label=f'Optimal Degree = {best_degree}')

# Rough labels for underfitting / overfitting regions
max_err = max(max(train_errors), max(test_errors))

if best_degree > 1:
    plt.text(1.05,
             max_err * 0.9,
             'Underfitting\n(high bias)',
             ha='left', va='top')

if best_degree < max(degrees):
    plt.text(best_degree + 0.5,
             max_err * 0.9,
             'Overfitting\n(high variance)',
             ha='left', va='top')

plt.legend()
plt.show()

# ---------------------------------------------------------------
# BONUS — k-fold Cross-Validation over Model Degree
# ---------------------------------------------------------------
# We will use k-fold (e.g., k=5) cross-validation on the *whole* dataset
# to estimate generalization error for each polynomial degree.

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

cv_means = []
cv_stds = []

for d in degrees:
    # Build a pipeline: PolynomialFeatures -> LinearRegression
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=d, include_bias=False)),
        ("linreg", LinearRegression())
    ])

    # cross_val_score uses negative MSE for 'neg_mean_squared_error'
    scores = cross_val_score(
        model,
        X,
        y,
        cv=kf,
        scoring='neg_mean_squared_error'
    )

    # Convert negative MSE back to positive MSE
    mse_scores = -scores
    cv_means.append(mse_scores.mean())
    cv_stds.append(mse_scores.std())

    print(f"[CV] Degree {d:2d} | Mean MSE: {mse_scores.mean():.4f} | Std: {mse_scores.std():.4f}")

# Plot cross-validation error vs degree
plt.figure(figsize=(9, 5))
plt.errorbar(
    degrees,
    cv_means,
    yerr=cv_stds,
    marker='o',
    capsize=4,
    label=f'{k}-fold CV Error (MSE)'
)

plt.xlabel('Model Complexity (Polynomial Degree)')
plt.ylabel('Cross-Validated Mean Squared Error')
plt.xticks(degrees)
plt.title(f'{k}-fold Cross-Validation Error vs Polynomial Degree')
plt.grid(True)
plt.legend()
plt.show()

# ---------------------------------------------------------------
# Step 4 — Discussion: Short Written Answers
# ---------------------------------------------------------------
# These answers are written in general terms, but they refer to the
# best_degree found above and to the shapes of the curves.

discussion = f"""\
STEP 4 – DISCUSSION

1) Which polynomial degree gives the best generalization?

   From the test MSE curve, the polynomial degree that gives the best
   generalization is degree = {{best_degree}}. At this degree, the test
   error is approximately minimal, which indicates a good balance between
   model complexity (flexibility) and the amount of data available.

2) How do the training and testing errors change as the degree increases?

   As the polynomial degree increases, the training error decreases
   monotonically or at least does not increase, because a more flexible
   model can always fit the training data at least as well as a simpler one.
   The testing error, however, typically first decreases (as underfitting
   is reduced) and then starts to increase again after around degree
   {{best_degree}}, because the model begins to overfit the noise in the
   training set. This characteristic “U–shaped” behavior of the test error
   is what we see in the validation curve.

3) Explain the observed behavior in terms of bias and variance.

   For very low polynomial degrees, the model is too simple to capture the
   true relationship between the features (T, RH, AH) and CO(GT). This leads
   to high bias: both training and testing errors are relatively large, and
   the model underfits. As we increase the degree towards {{best_degree}},
   the bias decreases and the model fits the data better without yet having
   very high variance, so the test error drops. For degrees higher than
   {{best_degree}}, the variance dominates: the model becomes very sensitive
   to small fluctuations and noise in the training data, fitting noise as if
   it were signal. This increases the test error and corresponds to
   overfitting (high variance, lower bias).

4) Comment on how sensor noise and missing data might affect this tradeoff.

   In this air quality dataset, sensor measurements (for CO, temperature,
   humidity, etc.) can be noisy or occasionally faulty. Noise effectively
   increases the variance of the target around the true underlying
   relationship. Highly flexible models (high-degree polynomials) tend to
   chase this noise, which makes overfitting worse and shifts the “safe”
   complexity range to lower degrees. Missing values (such as those coded
   as -200) reduce the amount of usable data once we drop rows with NaNs.
   With fewer effective samples, the variance of complex models increases
   further. Correctly handling missing data (e.g., by discarding or
   imputing) and not letting the degree grow too large is therefore crucial
   to obtaining a stable model that generalizes well.
"""

print(discussion)
